In [1]:
import pandas as pd

In [2]:
df_adni_merge = pd.read_csv("../data/clinical/ADNIMERGE_10Nov2025.csv")
df_adni_merge.head()

C:\Users\hp\AppData\Local\Temp\ipykernel_12708\2640617490.py:1: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  df_adni_merge = pd.read_csv("../data/clinical/ADNIMERGE_10Nov2025.csv")


,RID,COLPROT,ORIGPROT,PTID,SITE,VISCODE,EXAMDATE,DX_bl,AGE,PTGENDER,...,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,FBB_bl,Years_bl,Month_bl,Month,M,update_stamp
0,2,ADNI1,ADNI1,011_S_0002,11,bl,2005-09-08,CN,74.3,Male,...,NaN,1.33615,NaN,NaN,NaN,0.000000,0.00000,0,0,2023-07-07 04:59:40.0
1,3,ADNI1,ADNI1,011_S_0003,11,bl,2005-09-12,AD,81.3,Male,...,22.83,1.10860,NaN,NaN,NaN,0.000000,0.00000,0,0,2023-07-07 04:59:40.0
2,3,ADNI1,ADNI1,011_S_0003,11,m06,2006-03-13,AD,81.3,Male,...,22.83,1.10860,NaN,NaN,NaN,0.498289,5.96721,6,6,2023-07-07 04:59:40.0
3,3,ADNI1,ADNI1,011_S_0003,11,m12,2006-09-12,AD,81.3,Male,...,22.83,1.10860,NaN,NaN,NaN,0.999316,11.96720,12,12,2023-07-07 04:59:40.0
4,3,ADNI1,ADNI1,011_S_0003,11,m24,2007-09-12,AD,81.3,Male,...,22.83,1.10860,NaN,NaN,NaN,1.998630,23.93440,24,24,2023-07-07 04:59:40.0


In [4]:
df_adni_merge.info

<bound method DataFrame.info of         RID COLPROT ORIGPROT        PTID  SITE VISCODE    EXAMDATE DX_bl  \
0         2   ADNI1    ADNI1  011_S_0002    11      bl  2005-09-08    CN   
1         3   ADNI1    ADNI1  011_S_0003    11      bl  2005-09-12    AD   
2         3   ADNI1    ADNI1  011_S_0003    11     m06  2006-03-13    AD   
3         3   ADNI1    ADNI1  011_S_0003    11     m12  2006-09-12    AD   
4         3   ADNI1    ADNI1  011_S_0003    11     m24  2007-09-12    AD   
...     ...     ...      ...         ...   ...     ...         ...   ...   
16416  4349   ADNI3    ADNI2  018_S_4349    18    m138  2023-03-30    CN   
16417  6801   ADNI3    ADNI3  041_S_6801    41     m42  2023-06-30   SMC   
16418  5097   ADNI3    ADNI2  041_S_5097    41    m126  2023-08-16   SMC   
16419  6515   ADNI3    ADNI3  007_S_6515     7     m60  2023-08-24   SMC   
16420  5214   ADNI3    ADNI2  126_S_5214   126    m120  2023-09-06   SMC   

        AGE PTGENDER  ...  PTAU_bl   FDG_bl PIB_bl AV45

In [6]:
df_adni_merge.describe().T

,count,mean,std,min,25%,50%,75%,max
RID,16421.0,2899.082273,2220.442824,2.0000,751.000000,2308.0000,4707.00000,7125.000
SITE,16421.0,81.733329,132.762777,2.0000,22.000000,41.0000,123.00000,941.000
AGE,16412.0,73.214173,7.028099,50.4000,68.500000,73.2000,78.10000,91.400
PTEDUCAT,16421.0,16.105779,2.762672,4.0000,14.000000,16.0000,18.00000,20.000
APOE4,16056.0,0.516193,0.646953,0.0000,0.000000,0.0000,1.00000,2.000
...,...,...,...,...,...,...,...,...
FBB_bl,1099.0,1.163109,0.236850,0.8702,0.995100,1.0417,1.31860,1.909
Years_bl,16421.0,2.738280,2.939675,0.0000,0.506502,1.9603,4.01095,16.961
Month_bl,16421.0,32.792027,35.203809,0.0000,6.065570,23.4754,48.03280,203.115
Month,16421.0,32.712259,35.193789,0.0000,6.000000,24.0000,48.00000,204.000
